## Preprocessing

In [1]:
import warnings
import joblib
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
warnings.filterwarnings("ignore", category=FutureWarning, )

def translate_to_english(city_name):
    city_mapping = {
        'Bakı': 'Baku',
        'Gəncə': 'Ganja',
        'Sumqayıt': 'Sumgait',
        'Lənkəran': 'Lankaran',
        'Şamaxı': 'Shamakhi',
        'Sabirabad': 'Sabirabad',
        'Salyan': 'Salyan',
        'Masallı': 'Masalli',
        'Şirvan': 'Shirvan',
        'İsmayıllı': 'Ismayilli',
        'Biləsuvar': 'Bilasuvar',
        'Ağdaş': 'Agdash',
        'Tovuz': 'Tovuz',
        'Goranboy': 'Goranboy',
        'Şəki': 'Sheki',
        'Xırdalan': 'Khirdalan',
        'Ağcabədi': 'Agjabadi',
        'Quba': 'Quba',
        'Balakən': 'Balakan',
        'Şəmkir': 'Shamkir',
        'Qazax': 'Qazakh',
        'Mingəçevir': 'Mingachevir',
        'Bərdə': 'Barda',
        'Saatlı': 'Saatli',
        'Xaçmaz': 'Khachmaz',
        'Kürdəmir': 'Kurdamir',
        'Göyçay': 'Goychay',
        'Neftçala': 'Neftchala',
        'Ağsu': 'Agsu',
        'Qəbələ': 'Qabala',
        'Tərtər': 'Tartar',
        'Naxçıvan': 'Nakhchivan',
        'Astara': 'Astara',
        'Qax': 'Qakh',
        'Siyəzən': 'Siyezen',
        'Lerik': 'Lerik',
        'Yevlax': 'Yevlakh',
        'Gədəbəy': 'Gedabay',
        'İmişli': 'Imishli',
        'Zaqatala': 'Zaqatala',
        'Göygöl': 'Goygol',
        'Yardımlı': 'Yardimli',
        'Şabran': 'Shabran',
        'Qusar': 'Qusar',
        'Beyləqan': 'Beylagan',
        'Samux': 'Samukh',
        'Ağdam': 'Agdam',
        'Cəlilabad': 'Jalilabad',
        'Ağstafa': 'Agstafa',
        'Hacıqabul': 'Hajigabul',
        'Ucar': 'Ujar',
        'Füzuli': 'Fuzuli',
        'Qobustan': 'Gobustan',
        'Xudat': 'Khudat',
        'Oğuz': 'Oguz',
        'Zərdab': 'Zardab',
        'Babək': 'Babek',
        'Göytəpə': 'Goytepe',
        'Cəbrayıl': 'Jebrail',
        'Daşkəsən': 'Dashkasan',
        'Naftalan': 'Naftalan',
        'Dəliməmmədli': 'Delimammadli',
        'Şərur': 'Sharur',
        'Horadiz': 'Horadiz',
        'Ordubad': 'Ordubad',
        'Xızı': 'Khizi',
    }
    return city_mapping.get(city_name, city_name)

def extract_engine_power(data, column_name, new_column_name):
    data[new_column_name] = data[column_name].str.extract(r'(\d+\.\d+)').astype(float)
    return data

def extract_ride_km(data, column_name, new_column_name):
    data[new_column_name] = data[column_name].str.replace('[^\d.]', '', regex=True).astype(float)
    return data

def map_values(data, column_name, mapping):
    data[column_name] = data[column_name].apply(lambda value: mapping.get(value, value))
    return data





def separate_price_and_currency(data):
    def helper(price_str):
        price_str = price_str.replace(',', '')
        currency_codes = ['USD', 'EUR', 'GBP', 'AZN', 'JPY', 'RUB']
        for code in currency_codes:
            if code in price_str:
                currency = code
                price = price_str.replace(code, '').strip()
                return price, currency
        return price_str, 'AZN'

    data[['car_price', 'currency']] = data['price'].apply(helper).apply(pd.Series)
    data['car_price'] = pd.to_numeric(data['car_price'].str.replace('[^\d.]', '', regex=True), errors='coerce')
    data['car_price'] = data.apply(lambda row: row['car_price'] * 1.7 if row['currency'] == 'USD' else (
        row['car_price'] * 1.8 if row['currency'] == 'EUR' else row['car_price']), axis=1)
    return data

def preprocess_data(data):
    data['city'] = data['city'].apply(translate_to_english)
    data = map_values(data, 'ban_type', {
        'Offroader / SUV': 'SUV',
        'Sedan': 'Sedan',
        'Hetçbek': 'Hatchback',
        'Universal': 'Station Wagon',
        'Liftbek': 'Liftback',
        'Yük maşını': 'Truck',
        'Furqon': 'Van',
        'Minivan': 'Minivan',
        'Kupe': 'Coupe',
        'Motosiklet': 'Motorcycle',
        'Pikap': 'Pickup',
        'Dartqı': 'Convertible',
        'Mikroavtobus': 'Microbus',
        'Moped': 'Moped',
        'Avtobus': 'Bus',
        'Kabriolet': 'Convertible',
        'Van': 'Van',
        'Rodster': 'Roadster',
        'Kvadrosikl': 'Quad Bike'
    })
    data = map_values(data, 'colour', {
        'Ağ': 'Silver',
        'Qara': 'Black',
        'Gümüşü': 'Silver',
        'Göy': 'Blue',
        'Yaş Asfalt': 'Gray',
        'Boz': 'Brown',
        'Tünd qırmızı': 'Dark_Red',
        'Qırmızı': 'Red',
        'Yaşıl': 'Green',
        'Mavi': 'Blue',
        'Bej': 'Beige',
        'Qızılı': 'Gold',
        'Qəhvəyi': 'Brown',
        'Narıncı': 'Orange',
        'Sarı': 'Yellow',
        'Bənövşəyi': 'Purple',
        'Çəhrayı': 'Pink'
    })
    data = map_values(data, 'transmission', {
        'Avtomat': 'Automatic',
        'Mexaniki': 'Manual',
        'Variator': 'CVT',
        'Robotlaşdırılmış': 'Automated_Manual'
    })
    data = map_values(data, 'gear', {
        'Tam': 'Full',
        'Ön': 'Front',
        'Arxa': 'Rear'
    })
    data = map_values(data, 'is_new', {
        'Bəli': 'Yes',
        'Xeyr': 'No'
    })
    data = extract_engine_power(data, 'engine', 'engine_power')
    data = extract_ride_km(data, 'ride', 'ride_km')
    data = separate_price_and_currency(data)
    return data

def split_data(data, original_features, target, test_size=0.2, random_state=47):
    data = data.dropna()
    data[original_features].to_excel('preprocessed_data.xlsx', index=False)
    data_copy = pd.get_dummies(data[original_features + [target]], drop_first=True)
    X = data_copy.drop(columns=[target])
    y = data_copy[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

def train_xgboost_model(X_train, y_train, learning_rate=0.4, max_depth=4, n_estimators=1200):
    model = XGBRegressor(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators)
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    print("Model performance metrics")
    print("-----------------------")
    print(f"R-squared: {r2:.2f}")
    print(f"Root Mean Squared Error: {rmse:.2f}")
    print(f"Mean Absolute Error: {mae:.2f}")
    print("-----------------------")
    return y_pred


data_1 = pd.read_csv('turboaz_27_09_2023.csv')
data_2 = pd.read_csv('08112023.csv')
data = preprocess_data(pd.concat([data_1, data_2]).drop_duplicates().dropna())
original_features = ['city', 'make', 'model', 'year', 'ban_type', 'colour', 'engine_power', 'ride_km', 'transmission','gear', 'is_new']
target = 'car_price'
X_train, X_test, y_train, y_test = split_data(data, original_features, target)
model = train_xgboost_model(X_train, y_train)
y_pred = evaluate_model(model, X_test, y_test)
joblib.dump(model, 'xgboost_model.pkl')


Model performance metrics
-----------------------
R-squared: 0.96
Root Mean Squared Error: 10251.54
Mean Absolute Error: 4230.39
-----------------------


['xgboost_model.pkl']

## Lazypredict

In [36]:
import warnings
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
warnings.filterwarnings("ignore", category=FutureWarning)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)
reg = LazyRegressor(ignore_warnings=True, custom_metric=None)
models = reg.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 42/42 [02:18<00:00,  3.29s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000287 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 17016, number of used features: 3
[LightGBM] [Info] Start training from score 40383.840330


(                               Adjusted R-Squared  R-Squared       RMSE  \
 Model                                                                     
 LGBMRegressor                                0.72       0.82   22349.46   
 ExtraTreesRegressor                          0.71       0.81   22628.17   
 HistGradientBoostingRegressor                0.71       0.81   22648.64   
 XGBRegressor                                 0.71       0.81   22676.82   
 RandomForestRegressor                        0.71       0.81   22682.79   
 GradientBoostingRegressor                    0.70       0.80   23058.56   
 BaggingRegressor                             0.69       0.80   23299.08   
 ExtraTreeRegressor                           0.67       0.79   24138.83   
 KNeighborsRegressor                          0.65       0.78   24682.99   
 DecisionTreeRegressor                        0.65       0.77   24819.13   
 AdaBoostRegressor                            0.48       0.66   30201.33   
 PoissonRegr

## Grid Search for LGBM

In [3]:
import warnings
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
warnings.filterwarnings("ignore", category=FutureWarning)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [6, 8, 10],
    'min_child_samples': [20, 50, 100],
}
lgbm = LGBMRegressor()
grid_search = GridSearchCV(lgbm, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Model performance metrics")
print("R-squared:", r2)
print("Root Mean Squared Error:", rmse)
# joblib.dump(best_model, 'best_lgbm_model.pkl')

Model performance metrics
R-squared: 0.9389794990017293
Root Mean Squared Error: 12881.07294853125


## Gridsearch for XGB

In [8]:
import warnings
import joblib
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)
param_grid = {
    'n_estimators': [ 300,400,500,600 ,700,800,900], 
    'learning_rate': [0.3, 0.4, 0.5,0.6,0.7,0.8,0.9],  
    'max_depth': [3, 4, 5 ,6,7,8,9], 
}
xgb_regressor = XGBRegressor()
grid_search = GridSearchCV(xgb_regressor, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
print("Model performance metrics")
print("-----------------------")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"R-squared: {r2:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print("-----------------------")
# joblib.dump(best_model, 'xgboost_best_model.pkl')

Model performance metrics
-----------------------
Best Parameters: {'learning_rate': 0.4, 'max_depth': 4, 'n_estimators': 900}
R-squared: 0.95
Root Mean Squared Error: 11333.76
Mean Absolute Error: 4381.91
-----------------------


## XGB

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)
model = XGBRegressor(learning_rate = 0.4 ,max_depth=4 ,n_estimators = 1600  )
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
print("Model perforamnce metrics")
print("-----------------------")
print(f"R-squared: {r2:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print("-----------------------")
joblib.dump(model, 'xgboost_model.pkl')

Model perforamnce metrics
-----------------------
R-squared: 0.95
Root Mean Squared Error: 11318.41
Mean Absolute Error: 4230.30
-----------------------


['xgboost_model.pkl']